In [ ]:
import tensorflow as tf
from numpy.random import seed
seed(42)
tf.random.set_seed(42)

In [ ]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from skimage.transform import resize
from scipy.ndimage import zoom
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.losses import *
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.initializers import *
from keras import metrics
from keras import optimizers
import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
%matplotlib inline

In [ ]:
from evaluation_metrics import change_to_3D_MRI, calculate_dice, calculate_metrics, calculate_stats_3D_data

## Load data

In [ ]:
X_train = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/normalized_data/X_train.npy', allow_pickle = True)
X_val = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/normalized_data/X_val.npy', allow_pickle = True)
X_test = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/normalized_data/X_test.npy', allow_pickle = True) 
y_train = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/normalized_data/y_train.npy', allow_pickle = True)
y_val = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/normalized_data/y_val.npy', allow_pickle = True)
y_test = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/normalized_data/y_test.npy', allow_pickle = True)


print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# specify the lenght of the sequence that you use

n = 6

train_sequence = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/labels/train_sequence.npy', allow_pickle = True)
val_sequence = np.load('/content/drive/My Drive/Colab Notebooks/loading_data/labels/val_sequence.npy', allow_pickle = True)
test_sequence =  np.load('/content/drive/My Drive/Colab Notebooks/loading_data/labels/test_sequence.npy', allow_pickle = True)

print(train_sequence.shape)
print(val_sequence.shape)
print(test_sequence.shape)

In [ ]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    coef = (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return coef

In [ ]:
def loss(y_true, y_pred):
    def dice_coef_loss(y_true, y_pred):
        return 1-dice_coef(y_true, y_pred)

    return binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

In [ ]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

## RFCN-2

In [ ]:
# try this

input_size = (4,64,64)
input_sequence = ((n,64,64,1))

inputs = Input(input_size)
conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(inputs)
conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv1)
batch1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch1)

conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool1)
conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv2)
batch2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch2)

conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool2)
conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv3)
batch3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch3)


conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool3)
conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv5)
batch5 = BatchNormalization()(conv5)
drop5 = Dropout(0.5)(batch5)

up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(drop5))
merge7 = concatenate([conv3,up7], axis = 1)
conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge7)
conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv7)
batch7 = BatchNormalization()(conv7)

up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(batch7))
merge8 = concatenate([conv2,up8], axis = 1)
conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge8)
conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv8)
batch8 = BatchNormalization()(conv8)

up9 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(batch8))
merge9 = concatenate([conv1,up9], axis = 1)
conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge9)

input_seq = Input(input_sequence)
convLSTM = ConvLSTM2D(64, 3, data_format='channels_last', padding='same', kernel_initializer = 'he_normal')(input_seq)
perm1 = Permute((3,1,2))(convLSTM)
conv4 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(perm1)
merge_seq = concatenate([conv4, conv9], axis = 1)
conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge_seq)



conv10 = Conv2D(1, 1, activation = 'sigmoid', data_format = 'channels_first')(conv9)



model = Model(input = [inputs, input_seq], output = conv10)

model.compile(optimizer = Adam(lr = 0.001), loss = loss , metrics = [dice_coef])

In [ ]:
model.summary()

In [ ]:
log_dir = "/content/drive/My Drive/Colab Notebooks/results/result_unet_GRU" + datetime.datetime.now().strftime('%Y%m%d')

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

weight_saver = ModelCheckpoint("/content/drive/My Drive/Colab Notebooks/results/result_unet_GRU/unet_GRU_weights.{epoch:02d}-{val_loss:.2f}.hdf5",
                               monitor = 'val_dice_coef', period=1,
                               save_best_only = False, mode = 'max', save_weights_only = True)

reduce_lr_on_plateau = ReduceLROnPlateau(monitor = 'val_dice_coef', factor = 0.5, patience = 5, verbose = 1, 
                                          mode = 'max', cooldown = 1, min_lr = 1e-7)

early = EarlyStopping(monitor = "val_dice_coef", mode = "max", patience = 30)

print(log_dir)

In [ ]:
result = model.fit([X_train, train_adv_seq_flip], y_train, batch_size=64, epochs=300, validation_data=([X_val, val_adv_seq_flip], y_val),
                   callbacks= [tensorboard_callback, weight_saver, reduce_lr_on_plateau, early])

## RFCN-1

In [ ]:
input_size = (4,64,64)
input_sequence = ((n,64,64,1))

inputs = Input(input_size)
conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(inputs)
conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv1)
batch1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch1)

conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool1)
conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv2)
batch2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch2)

conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool2)
conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv3)
batch3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch3)

conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool3)
conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv5)

input_seq = Input(input_sequence)
convLSTM = ConvLSTM2D(64, 3, data_format='channels_last', padding='same', kernel_initializer = 'he_normal')(input_seq)
perm1 = Permute((3,1,2))(convLSTM)
conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(perm1)
pool4 = MaxPool2D(pool_size=(8,8), data_format = 'channels_first')(conv4)
merge_seq = concatenate([pool4, conv6], axis = 1)

conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge_seq)
batch5 = BatchNormalization()(conv5)
drop5 = Dropout(0.5)(batch5)

up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(drop5))
merge7 = concatenate([conv3,up7], axis = 1)
conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge7)
conv7 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv7)
batch7 = BatchNormalization()(conv7)

up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(batch7))
merge8 = concatenate([conv2,up8], axis = 1)
conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge8)
conv8 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv8)
batch8 = BatchNormalization()(conv8)

up9 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(batch8))
merge9 = concatenate([conv1,up9], axis = 1)
conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge9)
conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv9)
conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv9)
conv10 = Conv2D(1, 1, activation = 'sigmoid', data_format = 'channels_first')(conv9)


model = Model(input = [inputs, input_seq], output = conv10)

model.compile(optimizer = Adam(lr = 0.001), loss = loss , metrics = [dice_coef, metrics.SpecificityAtSensitivity(0.5), metrics.SensitivityAtSpecificity(0.5) ])

In [ ]:
model.summary()

## Test results

In [ ]:
model.load_weights('RFCN-2-n6.30-0.15.hdf5')

In [ ]:
test_result = model.predict([X_test,test_sequence])
test_result_3D = change_to_3D_MRI(test_result)
y_test_3D = change_to_3D_MRI(y_test)
test_scores = calculate_stats_3D_data(test_result_3D, y_test_3D, labels_X_test)

In [ ]:
test_scores